In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from custom_models import cnn
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import time
import os
from PIL import Image
from torchvision.transforms import v2
from tempfile import TemporaryDirectory
import shutil
import random

cudnn.benchmark = True
plt.ion()   # interactive mode

#### Data Transformations

In [2]:
# data transformations to loop through
minimal_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

basic_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.RandomHorizontalFlip(p=0.5),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

auto_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.AutoAugment(policy=v2.AutoAugmentPolicy.IMAGENET),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

#### Create Training Set with Real/Synthetic Images

In [3]:
def makeSyntheticTrain(train_directory, synthetic_train_directory, train_percentage, synthetic_percentage):

    # Remove any existing images in directory
    try:
        shutil.rmtree(synthetic_train_directory)
    except:
        print("directory does not exist")

    # Loop through subfolders, generate synthetic images
    subfolders = [f for f in os.listdir(train_directory)]

    for s in subfolders:
        # for each subfolder in the train directory, make the same in the synthetic train directory
        os.makedirs(f"{synthetic_train_directory}/{s}", exist_ok=True)
        
        # get a random sample from each subfolder
        subfolder_path = f"{train_directory}/{s}"
        files = os.listdir(subfolder_path)
        sample_files = random.sample(files, round(len(files)*train_percentage))
        
        # create synthetic sample based on sampled original images
        synthetic_subfolder_path = subfolder_path.replace('train','synthetic')
        synthetic_files = [f for f in os.listdir(synthetic_subfolder_path) if int(f.replace('.png','').split('_')[1]) in [int(f.replace('.png','').split('_')[1]) for f in sample_files]]
        synthetic_sample_files = random.sample(synthetic_files, round(len(files)*synthetic_percentage))
        
        # Move sample files to synthetic directory
        for f in sample_files:
            
            image_path = f"{subfolder_path}/{f}"
            destination_directory = f"{synthetic_train_directory}/{s}/"
            shutil.copyfile(image_path, destination_directory + image_path.split('/')[-1])

        # Move synthetic sample files to synthetic directory
        for f in synthetic_sample_files:

            image_path = f"{synthetic_subfolder_path}/{f}"
            destination_directory = f"{synthetic_train_directory}/{s}/"
            shutil.copyfile(image_path, destination_directory + image_path.split('/')[-1])

#### Read Data

In [4]:
def get_data(data_dir, data_sets, data_transforms):
    
    image_datasets = {
        x: datasets.ImageFolder(
            os.path.join(data_dir, x),
            data_transforms[x]
        )
        for x in data_sets
    }

    dataloaders = {
        x: DataLoader(
            image_datasets[x], 
            batch_size=4,
            shuffle=True, 
            num_workers=4
        )
        for x in data_sets
    }

    dataset_sizes = {
        x: len(image_datasets[x]) 
        for x in data_sets
    }

    class_names = image_datasets['synthetic_train'].classes

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    return image_datasets, dataloaders, dataset_sizes, class_names, device

#### Train Model

In [5]:
def train_model(model, criterion, optimizer, dataloaders, dataset_sizes, num_epochs=10):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        
        best_test_loss = float('inf')
        patience = 2  # Number of epochs to wait for improvement before stopping
        test_losses = []
        train_losses = []
        test_acc = []
        train_acc = []
        patience_counter = 0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['synthetic_train', 'test']:
                if phase == 'synthetic_train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'synthetic_train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'synthetic_train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                
                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double().item() / dataset_sizes[phase]

                if phase =='synthetic_train':
                    train_losses.append(epoch_loss)
                    train_acc.append(epoch_acc)
                else:
                    test_losses.append(epoch_loss)
                    test_acc.append(epoch_acc)
                    
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'test' and epoch_loss <= best_test_loss:
                    best_test_loss = epoch_loss
                    patience_counter = 0  # Reset counter
                    torch.save(model.state_dict(), best_model_params_path)
                elif phase == 'test' and epoch_loss > best_test_loss:
                    patience_counter += 1
                
                # Early stopping check
                #if patience_counter >= patience:
                #    print("Stopping early due to no improvement in validation loss.")
                #    break

        # store results in dataframe
        dat = {
            "epoch": range(len(test_losses)),
            "test_losses": test_losses,
            "train_losses": train_losses,
            "test_accuracies": test_acc,
            "train_accuracies": train_acc
        }

        result = pd.DataFrame(data=dat)
        print()
        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best test Loss: {best_test_loss:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
    return result, model

#### Loop through different training scenarios

In [6]:
num_epochs = 10
num_sims = 10
train_percs = [1.0, 0.8, 0.8, 0.6, 0.6, 0.4, 0.4]
synth_percs = [0.0, 0.2, 0.0, 0.4, 0.0, 0.6, 0.0]
transforms = {
    'minimal': minimal_transforms,
    'basic': basic_transforms,
    'auto': auto_transforms
}

for train_percentage, synthetic_percentage in zip(train_percs, synth_percs):
    for active_transform in transforms.keys():
        df_all_results = pd.DataFrame()
        for n in range(num_sims):
    
            # make the synthetic training dataset
            makeSyntheticTrain(
                train_directory='../data/alzheimer_mri/train',
                synthetic_train_directory='../data/alzheimer_mri/synthetic_train', 
                train_percentage=train_percentage, 
                synthetic_percentage=synthetic_percentage
            )

            # get and load datasets
            data_dir = '../data/alzheimer_mri'
            data_sets = ['synthetic_train','test']
            image_datasets, dataloaders, dataset_sizes, class_names, device = get_data(
                data_dir=data_dir, 
                data_sets=data_sets, 
                data_transforms=transforms.get(active_transform)
            )

            # instantiate model
            model = cnn.CNN(in_channels=1, num_classes=4)

            # Set the size of each output sample to nn.Linear(num_ftrs, len(class_names))
            #num_ftrs = 4 #model.fc.in_features
            #model.fc = nn.Linear(num_ftrs, len(class_names))
            model = model.to(device)

            # train the model
            df_results,_ = model = train_model(
                model=model, 
                criterion = nn.CrossEntropyLoss(),
                optimizer = optim.Adam(model.parameters(), lr=0.001), 
                dataloaders=dataloaders,
                dataset_sizes=dataset_sizes,
                num_epochs=num_epochs
            )

            df_results['train_percentage'] = train_percentage
            df_results['synth_percentage'] = synthetic_percentage
            df_results['transform'] = active_transform
            df_results['sim_num'] = n
            df_results['category'] = df_results.apply(lambda row: row['transform']+'_'+str(row['train_percentage'])+'_'+str(row['synth_percentage']), axis=1)
            df_all_results = pd.concat([df_all_results, df_results],ignore_index=True)
        df_all_results.to_csv('../results/results_cnn_'+active_transform+'_'+str(train_percentage)+'_'+str(synthetic_percentage)+'.csv')

/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 0.8802 Acc: 0.5699
test Loss: 0.7105 Acc: 0.6805
Epoch 1/9
----------
synthetic_train Loss: 0.3843 Acc: 0.8420
test Loss: 0.6111 Acc: 0.7758
Epoch 2/9
----------
synthetic_train Loss: 0.1043 Acc: 0.9619
test Loss: 0.1858 Acc: 0.9539
Epoch 3/9
----------
synthetic_train Loss: 0.0456 Acc: 0.9848
test Loss: 0.1377 Acc: 0.9594
Epoch 4/9
----------
synthetic_train Loss: 0.0430 Acc: 0.9854
test Loss: 0.1902 Acc: 0.9516
Epoch 5/9
----------
synthetic_train Loss: 0.0235 Acc: 0.9926
test Loss: 0.1288 Acc: 0.9594
Epoch 6/9
----------
synthetic_train Loss: 0.0287 Acc: 0.9904
test Loss: 0.1407 Acc: 0.9617
Epoch 7/9
----------
synthetic_train Loss: 0.0275 Acc: 0.9920
test Loss: 0.1140 Acc: 0.9703
Epoch 8/9
----------
synthetic_train Loss: 0.0188 Acc: 0.9959
test Loss: 0.1315 Acc: 0.9664
Epoch 9/9
----------
synthetic_train Loss: 0.0252 Acc: 0.9914
test Loss: 0.1804 Acc: 0.9602

Training complete in 3m 24s
Best test Loss: 0.113975


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 0.8866 Acc: 0.5727
test Loss: 0.7928 Acc: 0.6398
Epoch 1/9
----------
synthetic_train Loss: 0.3382 Acc: 0.8658
test Loss: 0.2741 Acc: 0.9000
Epoch 2/9
----------
synthetic_train Loss: 0.0807 Acc: 0.9705
test Loss: 0.2054 Acc: 0.9281
Epoch 3/9
----------
synthetic_train Loss: 0.0445 Acc: 0.9846
test Loss: 0.2231 Acc: 0.9359
Epoch 4/9
----------
synthetic_train Loss: 0.0580 Acc: 0.9824
test Loss: 0.1015 Acc: 0.9727
Epoch 5/9
----------
synthetic_train Loss: 0.0263 Acc: 0.9910
test Loss: 0.1948 Acc: 0.9492
Epoch 6/9
----------
synthetic_train Loss: 0.0270 Acc: 0.9893
test Loss: 0.1341 Acc: 0.9711
Epoch 7/9
----------
synthetic_train Loss: 0.0317 Acc: 0.9896
test Loss: 0.1411 Acc: 0.9680
Epoch 8/9
----------
synthetic_train Loss: 0.0191 Acc: 0.9947
test Loss: 0.1760 Acc: 0.9492
Epoch 9/9
----------
synthetic_train Loss: 0.0259 Acc: 0.9910
test Loss: 0.1193 Acc: 0.9766

Training complete in 3m 30s
Best test Loss: 0.101473


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0559 Acc: 0.4902
test Loss: 0.9874 Acc: 0.4953
Epoch 1/9
----------
synthetic_train Loss: 0.9999 Acc: 0.5191
test Loss: 0.9510 Acc: 0.5586
Epoch 2/9
----------
synthetic_train Loss: 0.9626 Acc: 0.5381
test Loss: 1.0271 Acc: 0.5172
Epoch 3/9
----------
synthetic_train Loss: 0.9097 Acc: 0.5660
test Loss: 0.7492 Acc: 0.6773
Epoch 4/9
----------
synthetic_train Loss: 0.8367 Acc: 0.6152
test Loss: 0.5992 Acc: 0.7266
Epoch 5/9
----------
synthetic_train Loss: 0.7327 Acc: 0.6701
test Loss: 0.4539 Acc: 0.8156
Epoch 6/9
----------
synthetic_train Loss: 0.6662 Acc: 0.7012
test Loss: 0.4117 Acc: 0.8367
Epoch 7/9
----------
synthetic_train Loss: 0.6129 Acc: 0.7301
test Loss: 0.3308 Acc: 0.8719
Epoch 8/9
----------
synthetic_train Loss: 0.5558 Acc: 0.7480
test Loss: 0.2582 Acc: 0.9047
Epoch 9/9
----------
synthetic_train Loss: 0.5188 Acc: 0.7730
test Loss: 0.1953 Acc: 0.9281

Training complete in 3m 37s
Best test Loss: 0.195306


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0501 Acc: 0.4895
test Loss: 0.9748 Acc: 0.5508
Epoch 1/9
----------
synthetic_train Loss: 0.9929 Acc: 0.5168
test Loss: 0.8599 Acc: 0.5758
Epoch 2/9
----------
synthetic_train Loss: 0.9525 Acc: 0.5371
test Loss: 0.8205 Acc: 0.6148
Epoch 3/9
----------
synthetic_train Loss: 0.9229 Acc: 0.5582
test Loss: 0.7050 Acc: 0.6789
Epoch 4/9
----------
synthetic_train Loss: 0.8513 Acc: 0.6016
test Loss: 0.5863 Acc: 0.7547
Epoch 5/9
----------


: 